# 0. Configuración de librerias y requisitos previos

In [11]:
"""
Se importan las librerías necesarias para el procesamiento de texto y manejo de archivos.

- nltk.tokenize.regexp_tokenize: para tokenización basada en expresiones regulares.
- nltk.stem.SnowballStemmer: para aplicar stemming en inglés.

- xml.etree.ElementTree: para parseo de archivos en formato XML.

- collections: para estructuras de datos con valores por defecto y operaciones de conteo.

- os: para operaciones relacionadas con el sistema de archivos.

- typing: para anotaciones de tipo.
"""

from nltk.tokenize import regexp_tokenize
from nltk.stem import SnowballStemmer

import xml.etree.ElementTree as ET

from collections import Counter, defaultdict

from typing import List, Dict, Union, Set

import os

In [12]:
"""
Carga de stopwords desde un archivo local.

- Evita la descarga de NLTK y permite usar una lista personalizada de palabras vacías.
- Se construye el conjunto `stop_words` para filtrar palabras vacías durante el preprocesamiento.
"""

# NO cambie esta ruta
path_stopwords = "../data/stopwords/english"

def load_stopwords(file_path: str) -> Set[str]:
    
    """
    Carga stopwords desde un archivo de texto plano.

    Cada línea del archivo debe contener una palabra. La función convierte
    todas las palabras a minúsculas y elimina líneas vacías.

    Parámetros
    ----------
    file_path : str
        Ruta del archivo de stopwords.

    Retorna
    -------
    Set[str]
        Conjunto de palabras que deben considerarse stopwords.
    """
    
    with open(file_path, "r", encoding = "utf-8") as file:
        stop_words = {line.strip().lower() for line in file if line.strip()}
    return stop_words

stop_words = load_stopwords(path_stopwords)

In [13]:
"""
Configuración del stemmer y definición del patrón de tokenización.

- Se inicializa `SnowballStemmer("english")` para reducir las palabras a su raíz 
  durante el preprocesamiento.
- Se define la expresión regular `pattern` que controla cómo se segmenta el texto
  en tokens. Incluye reglas para:
    * Abreviaturas (e.g., U.S.A.)
    * Palabras con guiones internos (e.g., non-euclidean)
    * Monedas, números y porcentajes
    * Puntos suspensivos (...)
    * Signos de puntuación como tokens separados
"""

stemmer = SnowballStemmer("english")

pattern = r'''(?x)              
    (?:[A-Z]\.)+[A-Z]?          # abreviaturas, e.g. U.S.A.
  | [A-Za-z]+(?:-[A-Za-z]+)*    # palabras con guiones internos, e.g. non-euclidean
  | \$?\d+(?:\.\d+)?%?          # monedas, números, porcentajes
  | \.\.\.                      # puntos suspensivos
  | [][.,;"'?():_`-]            # puntuación como tokens separados
'''

# 1. Preprocesamiento del texto

In [14]:
def preprocess(text: str) -> List[str]:
    
	"""
    Preprocesa un texto aplicando tokenización, normalización, eliminación de stopwords y stemming.

    El proceso de preprocesamiento transforma un texto crudo en una lista de tokens
    limpios y estandarizados, listos para tareas de recuperación de información o NLP.

    Pasos:
        1. Tokenización: se divide el texto en palabras usando la expresión regular `pattern`.
        2. Normalización: se convierten todos los tokens a minúsculas.
        3. Eliminación de stopwords: se filtran palabras vacías presentes en `stop_words`.
        4. Stemming: se aplica `SnowballStemmer` para reducir los tokens a su raíz.

    Parámetros
    ----------
    text : str
        Texto de entrada a procesar.

    Retorna
    -------
    List[str]
        Lista de tokens preprocesados.
    """
      
	# Generación de tokens
	tokens = regexp_tokenize(text, pattern)
	# Normalización 
	tokens = [token.lower() for token in tokens]
	# Eliminación de palabras de parada
	tokens = [token for token in tokens if token not in stop_words]
	# Stemming 
	tokens = [stemmer.stem(token) for token in tokens]
	return tokens

In [15]:
"""
Ejemplo de ejecución del preprocesamiento de texto.

Se procesa un texto de ejemplo que incluye números, signos de puntuación y abreviaturas. 
"""

example_text = "Se deben invertir $12.000 millones en D.I.A ..."

print("--------** preprocess **--------\n")
print("Parametros:\n")
print("- Texto de entrada = ", example_text)
print("\nResultado:\n")
print("- Tokens procesados = ", preprocess(example_text))


--------** preprocess **--------

Parametros:

- Texto de entrada =  Se deben invertir $12.000 millones en D.I.A ...

Resultado:

- Tokens procesados =  ['se', 'deben', 'invertir', '$12.000', 'millon', 'en', 'd.i.a', '...']


# 2. Carga de documentos

In [16]:
"""
Ruta de los documentos a procesar.

- `path_docs` indica la carpeta donde se encuentran los archivos de texto crudos.
- Cambie esta ruta según la ubicación de sus documentos en su sistema.
"""

# Cambie esta ruta según la ubicación de los archivos en su sistema
path_docs = "../data/docs-raw-texts"

In [17]:
def load_documents(path_docs: str) -> Dict[str, List[str]]:

	"""
    Carga y preprocesa los documentos de una carpeta específica.

    La función recorre todos los archivos con extensión `.naf` en la ruta
    especificada, extrae el ID del documento, el título y el contenido crudo,
    concatena título y contenido, y aplica la función `preprocess` para 
    obtener los tokens preprocesados de cada documento.

    Parámetros
    ----------
    path_docs : str
        Ruta de la carpeta que contiene los archivos `.naf` a procesar.
        Cambie esta ruta según la ubicación de sus documentos.

    Retorna
    -------
    Dict[str, List[str]]
        Diccionario donde las claves son los `doc_id` de cada documento y
        los valores son listas de tokens preprocesados.
    """
	
	docs = {}
	for file_name in os.listdir(path_docs):
		# Solo se revisan los archivos con extensión correcta
		if not file_name.endswith(".naf"):
			continue
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	return docs

In [18]:
"""
Ejemplo de carga y preprocesamiento de documentos.

Se utiliza la función `load_documents` para cargar todos los archivos `.naf`
de la carpeta especificada en `path_docs`. Cada documento se preprocesa y se almacena
en un diccionario donde la clave es el `doc_id` y el valor es la lista de tokens.

Luego se muestra el contenido preprocesado de un documento específico
con ID "d006".
"""

documents = load_documents(path_docs)
example_id = "d006"

print("--------** load documents **--------\n")
print("Parametros:\n")
print("- Ruta de los documentos = ", path_docs)
print("\nResultado:\n")
print("- Documentos cargados = ", len(documents))
print("- Ejemplo documento " + example_id + " = ", documents[example_id])

--------** load documents **--------

Parametros:

- Ruta de los documentos =  ../data/docs-raw-texts

Resultado:

- Documentos cargados =  331
- Ejemplo documento d006 =  ['eugenio', 'beltrami', 'non-euclidian', 'geometri', 'eugenio', 'beltrami', 'non-euclidian', 'geometri', '.', 'eugenio', 'beltrami', '(', '1835', '-', '1900', ')', '.', 'novemb', '16', ',', '1835', ',', 'italian', 'mathematician', 'eugenio', 'beltrami', 'born', '.', 'notabl', 'work', 'concern', 'differenti', 'geometri', 'mathemat', 'physic', '.', 'work', 'note', 'especi', 'clariti', 'exposit', '.', 'first', 'prove', 'consist', 'non-euclidean', 'geometri', 'model', 'surfac', 'constant', 'curvatur', ',', 'pseudospher', '.', 'eugenio', 'beltrami', 'born', 'cremona', 'lombardi', ',', 'part', 'austrian', 'empir', ',', 'part', 'itali', '.', 'son', 'artist', 'paint', 'miniatur', ',', 'young', 'eugenio', 'certain', 'inherit', 'artist', 'talent', 'famili', ',', 'case', 'addit', 'mathemat', 'talent', 'would', 'acquir', ',', 'm

# 3. Representación vectorial Tf-IDf

In [ ]:
def build_tf_index(documents: Dict[str, List[str]]) -> Dict[str, Dict[str, int]]:
    
	"""
    Construye un índice de frecuencia de términos (TF) a partir de los documentos preprocesados.

    La función recorre todos los documentos proporcionados y cuenta la frecuencia de
    cada token dentro de cada documento. El resultado es un diccionario donde cada
    documento tiene un subdiccionario que representa la frecuencia de cada término.

    Parámetros
    ----------
    documents : Dict[str, List[str]]
        Diccionario donde las claves son los `doc_id` de los documentos y los valores
        son listas de tokens preprocesados de cada documento, tal como devuelve
        la función `load_documents`.

    Retorna
    -------
    Dict[str, Dict[str, int]]
        Diccionario donde las claves son los `doc_id` de los documentos y los valores
        son diccionarios que mapean cada token a su frecuencia dentro del documento.
        Por ejemplo: { "doc1": {"palabra1": 3, "palabra2": 5}, ... }.
    """
     
	tf_index = {}
	# Conteo de la frecuencia de los terminos dentro de cada documento
	for doc_id, tokens in documents.items():
		tf_index[doc_id] = dict(Counter(tokens))
	return tf_index

In [ ]:
"""
Ejemplo de construcción del índice de frecuencia de términos (TF).

Se utiliza la función `build_tf_index` para generar un índice que contiene
la frecuencia de cada token dentro de cada documento preprocesado en `documents`.

Luego se muestra el subdiccionario correspondiente a un documento específico
con ID "d006", mostrando cuántas veces aparece cada token en ese documento.
"""

tf_index = build_tf_index(documents)
example_id = "d006"

print("--------** build TF index **--------\n")
print("Parametros:\n")
print("- Número de documentos cargados = ", len(documents))
print("\nResultado:\n")
print("- Índice TF generado = ", tf_index)
print("- Ejemplo de frecuencias del documento " + example_id + " = ", tf_index[example_id])


--------** build TF index **--------

Parametros:

- Número de documentos cargados =  331

Resultado:

- Índice TF generado =  {'d210': {'joseph': 13, 'weizenbaum': 19, 'famous': 2, 'eliza': 9, '.': 34, '(': 4, '1923': 3, '-': 2, '2008': 2, ')': 4, 'photo': 1, ':': 10, 'ulrich': 1, 'hansen': 1, 'januari': 3, '8': 2, ',': 48, 'comput': 16, 'scientist': 2, 'pioneer': 2, 'natur': 2, 'languag': 3, 'process': 3, 'artifici': 3, 'intellig': 3, 'later': 2, 'becam': 2, 'one': 2, 'lead': 1, 'critic': 2, 'born': 2, '1966': 3, 'publish': 2, 'simpl': 3, 'program': 7, 'name': 2, 'involv': 1, 'user': 2, 'convers': 2, 'bore': 1, 'strike': 1, 'resembl': 1, 'psychologist': 1, 'berlin': 3, 'jewish': 1, 'parent': 1, 'abl': 4, 'escap': 1, 'nazi': 1, 'germani': 1, '1936': 1, 'emigr': 1, 'famili': 1, 'unit': 1, 'state': 3, 'start': 1, 'studi': 3, 'mathemat': 2, 'wayn': 2, 'univers': 6, 'detroit': 1, '1941': 1, 'howev': 3, 'interrupt': 1, 'war': 1, 'serv': 1, 'militari': 1, 'meteorolog': 1, 'servic': 1, 'air'